# Import data

In [1]:
import pandas as pd

apiControls= pd.read_json('Schemes/apiControls.json')
cisco = pd.read_csv('Schemes/Cisco.csv')
medinaEucs = pd.read_csv('Schemes/MEDINA_EUCS_ControlsMapping_Clean.csv')
secNumCloud = pd.read_csv('Schemes/secnumcloud_controlsParsedAndTranslatedEnglish.csv')
spanishEns = pd.read_csv('Schemes/SpanishENS.csv')

In [2]:
apiControls

,uuid,code,name,descriptionTitle,descriptionExtended,extraInfoTitle,extraInfoExtended,finalInfoTitle,finalInfoExtended,implementationGuidelines,createdDate,lastModifiedDate,category,assuranceLevel,requirements,sourceSimilarControls,targetSimilarControls,relatedMetrics
0,00000000-0000-0000-0000-000000000000,AAA-00,GENERIC CONTROL,This control behaves as a generic control.,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '00000000-0000-0000-0000-000000000000...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
1,0026165d-7fdd-4834-8033-292e16c493f7,IAM-02,MANAGEMENT OF USER ACCOUNTS,Policies and procedures for managing the diffe...,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '606d35a4-a217-4ed0-ad2b-3068ffe409b7...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
2,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,OIS-01,INFORMATION SECURITY MANAGEMENT SYSTEM,The CSP operates an information security manag...,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '0b71c590-13f7-4eac-a927-91e11c902fa5...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
3,01006ca1-2dd3-4e84-bdff-db0db40912a6,SIM-04,Duty of the users to report security incidents...,The Cloud Service Provider informs employees a...,None,None,Cloud customers ensure through suitable contro...,partially,The Cloud Service Provider should inform its e...,NaN,1710154800,1710154800,{'uuid': 'df962f42-d441-4dbe-8bb2-9ab44c286947...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
4,02a1ea32-5ebc-4e34-ac0f-8ed3fda33b5c,OPS-03,Capacity Management - Controlling of Resources,Depending on the capabilities of the respectiv...,None,Resources according to the possibilities of th...,Cloud customers ensure through suitable contro...,partially,The existence of tools for controlling resourc...,NaN,1710154800,1710154800,{'uuid': '376740c0-29c9-4ecc-992d-da93dfce9f57...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,fda6073b-0141-438f-885c-28408b77498a,OPS-21,"MANAGING VULNERABILITIES, MALFUNCTIONS AND ERR...",System components are hardened to reduce their...,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '5b95970d-9e05-45e7-900b-70f368169293...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
290,fdb3daf4-3f2b-43da-bfc9-b817b0912a1f,COM-01,"Identification of applicable legal, regulatory...","The legal, regulatory, self-imposed and contra...",None,The Cloud Service Provider´s documentation may...,None,no,"A continuous audit of contract specifications,...",NaN,1710154800,1710154800,{'uuid': 'd46f6a56-a6f2-45cd-afbb-5bfb18368ac1...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
291,fed57d83-4959-4e8e-b23a-5b1165425775,OPS-22,SEPARATION OF DATASETS IN THE CLOUD INFRASTRUC...,System components are hardened to reduce their...,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '5b95970d-9e05-45e7-900b-70f368169293...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN
292,fedb6f08-32ed-4bb6-9101-1061072673e6,CS-04,NETWORKS FOR ADMINISTRATION,Administrative and operational management duti...,None,None,None,None,None,NaN,1710154800,1710154800,{'uuid': '69e9f4a6-8c00-4a8a-9e65-819a9231c23e...,{'uuid': 'ffffffff-ffff-ffff-ffff-ffffffffffff...,NaN,NaN,NaN,NaN


In [3]:
cisco

,Domain Title,Control Title,Control Reference,Control Wording,Control Type,SOC TSC Common Criteria,SOC TSC Availability,SOC TSC Confidentiality,SOC TSC Privacy,ISO 27001 ISMS,...,ISMAP Control,PCI Control,Saudi CCC Control,EU Code of Conduct Control,NIST SP 800-171 Control,EUCS Basic Control,EUCS Substantial Control,EUCS High Control,SecNumCloud Control,IRAP Control
0,Audit Compliance,Control Self-Assessments,CCF 1,Independent Control self-assessments are perfo...,Process,"CC1.5, CC2.1, CC2.2, CC2.3, CC3.1, CC3.2, CC4....",NaN,NaN,NaN,"Clause 6.1.1(e), Clause 6.1.2, Clause 6.1.3, C...",...,"12.7.1, 13.1.1.7, 17.1.3, 18.1.1, 18.2.1, 18.2...","6.4.6, 10.1","2-16-P-4, 1-3-T-1-1, 1-9-4-2, 2-1-2, 2-1-4, 1-...","5.5.A, 5.5.B, 6.2.K, 6.3.A",3.12.1 Periodically assess the security contro...,"OSI-01.4, OIS-02.3, CO-01.1, CO-02.1, CO-03...","OIS-01.5, CO-01.2, CO-02.2, CO-03.3, CO-04.2","OIS-01.3, ISP-02.5, PS-05.4, OPS-19.4, CO-...","18.2.1.a, 18.2.1.b, 18.3.a, 18.4.a",1163 1563 1564 0810 0336 1493
1,Audit Compliance,Security Policy Audit,CCF 2,"At least quarterly, [The Organization] reviews...",Process,NaN,NaN,NaN,NaN,NaN,...,NaN,"12.11.a, 12.11.b, 12.11.1",NaN,NaN,NaN,OPS-10.1,"OPS-04.2, OPS-10.2, CS-08.3",NaN,12.1.a,NaN
2,Audit Compliance,Customer Audits,CCF 3,"If applicable, Systems documented procedures r...",Process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"5.5.E, 5.5.F, 5.5.A, 5.5.C",NaN,CO-02.1,NaN,CO-02.3,NaN,NaN
3,Audit Compliance,Audit Program,CCF 4,A three-year audit program is in place which d...,Process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.2.1.a,NaN
4,Audit Compliance,Review of Audit Plan,CCF 5,Management performs a review of the audit plan...,People,NaN,NaN,NaN,NaN,NaN,...,4.4.1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Vulnerability Detection,Microsoft Office Macros,CCF 708,Allowed and blocked Microsoft Office macro exe...,Technology,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1675, 1676"
709,Vulnerability Detection,Data Transfer,CCF 709,All data transferred from a SECRET or TOP SECR...,Process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"664, 675"
710,Vulnerability Detection,Vulnerability scanner,CCF 710,A vulnerability scanner is used at least daily...,Technology,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700
711,Vulnerability Detection,Security Txt File,CCF 711,A ‘security.txt’ file is hosted for all intern...,Technology,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1717


In [4]:
medinaEucs

,EUCS Category,EUCS Control (2022),Code,C5.2020 GERMANY,SecNumCloud FRANCE,ISO 27002,ISO 27017
0,A1 - Organisation of Information Security,OIS-01 - INFORMATION SECURITY MANAGEMENT SYSTEM,NaN,OIS-01,NaN,5.4\n5.21,NaN
1,NaN,OIS-02 - SEGREGATION OF DUTIES,OIS-02,OIS-04,6.1\n 6.2,5.3,CLD.6.3.1
2,NaN,OIS-03 - CONTACT WITH AUTHORITIES AND INTERES...,NaN,OIS-05,6.3\n 6.4,5.5\n5.6,CLD.6.3.1
3,NaN,OIS-04 - INFORMATION SECURITY IN PROJECT MANAG...,NaN,OIS-05\nOIS-6\nOIS-7,6.5,5.8,CLD.6.3.1
4,A2 - Information Security Policies,ISP-01 - GLOBAL INFORMATION SECURITY POLICY,NaN,OIS-02,5.2\n 5.1,5.1,NaN
...,...,...,...,...,...,...,...
115,A20 - Product Security,PSS-01 - ERROR HANDLING AND LOGGING MECHANISMS,NaN,PSS-04,NaN,8.15,NaN
116,NaN,PSS-02 - SESSION MANAGEMENT,NaN,PSS-06,NaN,8.16,NaN
117,NaN,PSS-03 - SOFTWARE DEFINED NETWORKING,NaN,PSS-10,NaN,8.20\n8.21,CLD.13.1.4
118,NaN,PSS-04 - IMAGES FOR VIRTUAL MACHINES AND CONT...,PSS-04,PSS-11,NaN,NaN,NaN


In [5]:
secNumCloud

,ID,Description,Description_EN
0,5.1.a,Le prestataire doit opérer la prestation à l'é...,The service provider must operate in the state...
1,5.1.b,Le prestataire doit appliquer le guide d'hygiè...,The service provider must apply the ANSSI comp...
2,5.2.a,Le prestataire doit documenter et mettre en œu...,The service provider must document and impleme...
3,5.2.b,La politique de sécurité de l'information doit...,The information security policy must identify ...
4,5.2.c,La politique de sécurité de l'information doit...,The information security policy must in partic...
...,...,...,...
282,3.2.e,"Le commanditaire peut, conformément au process...","The sponsor may, in accordance with the qualif..."
283,3.2.f,La qualification d'un prestataire n'atteste pa...,The qualification of a service provider does n...
284,3.2.g,La qualification d'un prestataire n'atteste pa...,The qualification of a service provider does n...
285,3.2.h,La conformité du service du prestataire au réf...,The compliance of the service provider at the ...


In [6]:
spanishEns

,Control ID,Description
0,org.1.1,The objectives or mission of the organization.
1,org.1.2,The legal and regulatory framework within whic...
2,org.1.3,"Security roles or functions, defining for each..."
3,org.1.4,The structure of the security management and c...
4,org.1.5,Guidelines for the structuring of system secur...
...,...,...
204,mp.s.3.4,Web address resolution and connection informat...
205,mp.s.3.5,The organization in general and the workstatio...
206,mp.s.3.6,It shall protect against the action of harmful...
207,mp.s.3.7,An executive cookie control policy shall be es...


# Analizing relations

In [7]:
import pandas as pd
import plotly.graph_objects as go
import igraph as ig

# Creazione del grafo con igraph
g = ig.Graph(directed=True)

# Aggiunta nodi (dataframe)
dataframes = ["apiControls", "cisco", "medinaEucs", "secNumCloud", "spanishEns"]
g.add_vertices(dataframes)
g.vs["label"] = dataframes

# Aggiunta relazioni (edge) con descrizioni
relations = [
    ("cisco", "medinaEucs", "EUCS Control Codes"),
    ("medinaEucs", "secNumCloud", "SecNumCloud IDs"),
    ("cisco", "secNumCloud", "SecNumCloud Control"),
    ("medinaEucs", "cisco", "ISO 27002 References"),
    ("apiControls", "cisco", "Control Names/IDs")
]
for src, dst, label in relations:
    g.add_edge(g.vs.find(src).index, g.vs.find(dst).index, label=label)

# Calcola il layout del grafo
layout = g.layout("kk")
pos = {name: layout[i] for i, name in enumerate(g.vs["label"])}

# Crea liste per le posizioni dei nodi
edge_x = []
edge_y = []
for edge in g.es:
    start_node = g.vs[edge.source]["label"]
    end_node = g.vs[edge.target]["label"]
    x0, y0 = pos[start_node]
    x1, y1 = pos[end_node]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Traccia gli archi
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Traccia i nodi
node_x = [pos[name][0] for name in dataframes]
node_y = [pos[name][1] for name in dataframes]
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=dataframes,
    textposition="bottom center",
    textfont=dict(size=14),
    marker=dict(
        showscale=False,
        size=25,
        color='#FFDDC1',
        line=dict(color='black', width=1.5)))

# Crea le etichette degli archi
edge_labels = []
for edge in g.es:
    start_node = g.vs[edge.source]["label"]
    end_node = g.vs[edge.target]["label"]
    label = edge["label"]
    x0, y0 = pos[start_node]
    x1, y1 = pos[end_node]
    edge_labels.append(go.Scatter(
        x=[(x0+x1)/2], y=[(y0+y1)/2],
        mode='text',
        text=f'<b>{label}</b>',  # Modifica qui per il grassetto
        textposition="middle center",
        showlegend=False,
        textfont=dict(size=12, color="gray")))

# Combina tutti i tracciati
fig = go.Figure(data=[edge_trace, node_trace] + edge_labels,
                layout=go.Layout(
                    title={
                        'text': 'Relazioni tra DataFrame (Plotly)',
                        'font': {'size': 16}
                    },
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        ax=edge_x[i*3], ay=edge_y[i*3],
                        axref='x', ayref='y',
                        x=edge_x[i*3+1], y=edge_y[i*3+1],
                        xref='x', yref='y',
                        showarrow=True,
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=1,
                        arrowcolor='#888') for i in range(len(relations))],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()

# Clean and merge all data in one dataframe

In [8]:
import pandas as pd
import numpy as np

# Funzione migliorata per gestire valori multipli
def clean_split(column, delimiter=','):
    return column.fillna('').apply(
        lambda x: [item.strip() for item in str(x).split(delimiter) if item.strip()]
    )


# Preprocessamento apiControls
api_controls = apiControls[['uuid', 'code', 'name', 'descriptionExtended']].copy()

# Preprocessamento cisco - gestione colonne EUCS
cisco['EUCS_Combined'] = (
    cisco.get('EUCS Basic Control', pd.Series('')).fillna('') + ',' + 
    cisco.get('EUCS Substantial Control', pd.Series('')).fillna('') + ',' + 
    cisco.get('EUCS High Control', pd.Series('')).fillna('')
)
cisco['EUCS_Codes'] = clean_split(cisco['EUCS_Combined'])
cisco['SecNumCloud_Codes'] = clean_split(cisco.get('SecNumCloud Control', pd.Series('')))

# Preprocessamento medinaEucs - gestione nomi colonne alternativi
secnumcloud_col = next((col for col in medinaEucs.columns if 'SecNumCloud' in col), None)
medinaEucs['EUCS_Code'] = medinaEucs['EUCS Control (2022)'].str.extract(r'^(\w+-\d+)')

if secnumcloud_col:
    medinaEucs['SecNumCloud_FR'] = clean_split(medinaEucs[secnumcloud_col], delimiter='\\n')
else:
    medinaEucs['SecNumCloud_FR'] = [[] for _ in range(len(medinaEucs))]

# Creazione dataframe unificato
unified = []

for _, api_row in api_controls.iterrows():
    # 1. Trova corrispondenze in cisco per nome controllo
    if 'Control Title' in cisco.columns:
        cisco_matches = cisco[
            cisco['Control Title'].str.contains(api_row['name'], case=False, na=False)
        ]
    else:
        cisco_matches = pd.DataFrame()

    if not cisco_matches.empty:
        for _, cisco_row in cisco_matches.iterrows():
            # 2. Collega EUCS codes a medinaEucs
            for eucs_code in cisco_row['EUCS_Codes']:
                base_code = eucs_code.split('.')[0]  # Estrai codice base
                if 'EUCS_Code' in medinaEucs.columns:
                    medina_match = medinaEucs[medinaEucs['EUCS_Code'] == base_code]
                else:
                    medina_match = pd.DataFrame()

                if not medina_match.empty:
                    # 3. Collega SecNumCloud da medinaEucs
                    for sec_list in medina_match['SecNumCloud_FR']:
                        for sec_id in sec_list:
                            if 'ID' in secNumCloud.columns:
                                sec_match = secNumCloud[secNumCloud['ID'] == sec_id]
                                if not sec_match.empty:
                                    record = {
                                        'api_uuid': api_row['uuid'],
                                        'api_name': api_row['name'],
                                        'cisco_title': cisco_row.get('Control Title', ''),
                                        'eucs_code': eucs_code,
                                        'secNumCloud_id': sec_id,
                                        'description': f"{api_row.get('descriptionExtended', '')} | {cisco_row.get('Control Wording', '')} | {sec_match['Description_EN'].values[0]}",
                                        'source': 'cisco+medina+secNumCloud'
                                    }
                                    unified.append(record)

                # 4. Collega direttamente SecNumCloud da cisco
                for sec_id in cisco_row['SecNumCloud_Codes']:
                    if 'ID' in secNumCloud.columns:
                        sec_match = secNumCloud[secNumCloud['ID'] == sec_id]
                        if not sec_match.empty:
                            record = {
                                'api_uuid': api_row['uuid'],
                                'api_name': api_row['name'],
                                'cisco_title': cisco_row.get('Control Title', ''),
                                'eucs_code': eucs_code,
                                'secNumCloud_id': sec_id,
                                'description': f"{api_row.get('descriptionExtended', '')} | {cisco_row.get('Control Wording', '')} | {sec_match['Description_EN'].values[0]}",
                                'source': 'cisco+secNumCloud'
                            }
                            unified.append(record)

# Crea dataframe finale
if unified:
    unified_df = pd.DataFrame(unified).drop_duplicates()
    
    # Aggiunta eventuale di spanishEns (se si trovano collegamenti)
    # (da implementare in base alle esigenze specifiche)
    
    print(f"Dataframe unificato creato con {len(unified_df)} record")
    unified_df.to_csv('Output/unified_data.csv', index=False)

else:
    print("Nessun record trovato - verificare i dati di input")



/tmp/ipykernel_48181/995558024.py:39: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



Dataframe unificato creato con 160 record


In [9]:
unified_df

,api_uuid,api_name,cisco_title,eucs_code,secNumCloud_id,description,source
0,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,INFORMATION SECURITY MANAGEMENT SYSTEM,Privacy Information Security Management System...,RM-01.1,5.3.a,None | A documented Privacy Information Securi...,cisco+secNumCloud
1,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,INFORMATION SECURITY MANAGEMENT SYSTEM,Privacy Information Security Management System...,RM-01.1,5.3.b,None | A documented Privacy Information Securi...,cisco+secNumCloud
2,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,INFORMATION SECURITY MANAGEMENT SYSTEM,Privacy Information Security Management System...,RM-01.1,5.3.c,None | A documented Privacy Information Securi...,cisco+secNumCloud
3,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,INFORMATION SECURITY MANAGEMENT SYSTEM,Privacy Information Security Management System...,RM-01.2,5.3.a,None | A documented Privacy Information Securi...,cisco+secNumCloud
4,00a8fdd7-cafb-42bd-b2a4-2ae57211e789,INFORMATION SECURITY MANAGEMENT SYSTEM,Privacy Information Security Management System...,RM-01.2,5.3.b,None | A documented Privacy Information Securi...,cisco+secNumCloud
...,...,...,...,...,...,...,...
155,c7624b5b-781a-4890-b0cc-dc8c45bdf39a,Version Control,Version Control,CCM-02.3,14.2.c,Version control procedures provide appropriate...,cisco+secNumCloud
156,c7624b5b-781a-4890-b0cc-dc8c45bdf39a,Version Control,Version Control,CCM-03.9,14.2.c,Version control procedures provide appropriate...,cisco+secNumCloud
157,c7624b5b-781a-4890-b0cc-dc8c45bdf39a,Version Control,Version Control,CCM-06.2,14.2.c,Version control procedures provide appropriate...,cisco+secNumCloud
158,c7624b5b-781a-4890-b0cc-dc8c45bdf39a,Version Control,Version Control,CCM-06.3,14.2.c,Version control procedures provide appropriate...,cisco+secNumCloud
